# Reporte abonos cartolas BECH

In [43]:
import os
import datetime
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from openpyxl import Workbook

username = os.getlogin()            
hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
fecha_actual = hoy.strftime(date_formatexport)
fecha_cartola = hoy.strftime(date_format)

df_mes = pd.DataFrame
df_ajustes = pd.DataFrame
alm = pd.DataFrame

folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\IDENTIFICADOR\\AJUSTES'

print("Cargando archivos...")

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
df_list = []
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',
                        names=['monto','agencia','cierre'],
                        header=0)
    df_list.append(df)
df_cierres = pd.concat(df_list)


folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\CARTOLAS HISTORICAS'

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
df_list = []
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',
                        header=None,
                        thousands='.',
                        names=['fecha', 'sucursal', 'operacion', 'descripcion', 'cargos', 'abonos','ind'])
    df_list.append(df)
df_mes = pd.concat(df_list)
df_mes['fecha'] = df_mes['fecha'].str.replace('/', '-')
df_mes['fecha'] = pd.to_datetime(df_mes['fecha'],format='mixed',dayfirst=True)
df_mes = df_mes.sort_values(by='fecha')

df_agencias = df_cierres
df_agencias = df_agencias[df_agencias['monto'] != 0]
df_agencias = df_agencias.drop(columns=['cierre'])
df_agencias.sort_values(by='monto', ascending=True, inplace=True)
df_agencias.drop_duplicates(subset='monto',keep='first', inplace=True)


df_ident = pd.merge(df_mes, df_agencias, how='left', left_on='abonos', right_on='monto')
df_ident['agencia'] = df_ident['agencia'].astype(str).replace('\.0$','',regex=True)
df_ident['monto'] = df_ident['monto'].astype(str).replace('\.0$','',regex=True)

print("Archivos cargados")

Cargando archivos...
Archivos cargados


In [44]:
df_ident['fecha'] = df_ident['fecha'].dt.strftime(date_formatexport)
df_ident = df_ident.drop(columns=['ind','monto'])
df_ident.to_excel('C:\\Users\\' + username + '\\Documents\\Py\\JPNB\\GENERADOS\\Reporte mes ' + fecha_actual + '.xlsx',
            header=True,
            index=False)

# Obtener abonos segun glosa

Reemplazar texto a encontrar en descripción

In [22]:
df_muestra = df_mes[df_mes['descripcion'].str.contains('Transferencia', case=False)]
count = df_muestra.shape[0]
print (df_muestra)

         fecha        sucursal  operacion  \
728 2024-10-30  Stgo.principal    7020990   
729 2024-10-30  Stgo.principal    7081724   
730 2024-10-30  Stgo.principal    7003546   
731 2024-10-30  Stgo.principal    7088536   
732 2024-10-30  Stgo.principal    7052022   
..         ...             ...        ...   
58  2024-11-19  Stgo.principal    7074796   
57  2024-11-19  Stgo.principal    7017918   
55  2024-11-19  Stgo.principal    7093941   
54  2024-11-19  Stgo.principal       1077   
61  2024-11-19  Stgo.principal    7046504   

                                           descripcion  cargos  abonos ind  
728  Transferencia bancoestado de rut 9869958-9 gon...       0  224440   H  
729  Transferencia bancoestado de rut 8094669-4 lec...       0  225460   H  
730  Transferencia bancoestado de rut 14346368-0 go...       0  225540   H  
731  Transferencia bancoestado de rut 12370997-7 mo...       0  227390   H  
732  Transferencia bancoestado de rut 13043086-4 co...       0  227600   H

# Obtener cierres según monto

In [ ]:

df_muestra = df_ajustes.copy()
num = 317100
print("Monto cierre: "+ str(num))
df_muestra = df_ajustes[df_ajustes['mcca'] == num].tail()
if df_muestra.empty:
    print("No se encuentran cierres por el monto consultado")
else:
    df_muestra.drop_duplicates(subset='cierre',keep='first', inplace=True)
    print(df_muestra)

# Ubicar abono según N° Operacion

In [ ]:
df_muestra = df_mes[df_mes['operacion'].astype(str).str.contains('1562', case=False)].shape[0]
# sorted_df = count_df.sort_values(by='descripcion', ascending=False)
print (df_muestra)

# Montos de cierres por agencia

In [ ]:
num = 16047150
# df_muestra = df_agencias[df_agencias['mcca'].astype(str).str.contains(str(num), case=False)]
# sorted_df = count_df.sort_values(by='descripcion', ascending=False)
# print (df_muestra)

df_muestra = df_agencias.copy()
num = 18122320
print("Agencia: "+ str(num))
df_muestra = df_agencias[df_agencias["agencia"] == num]
if df_muestra.empty:
    print("No se encuentran registros de la agencia")
else:
    df_muestra.drop_duplicates(subset='mcca',keep='first')
    print(df_muestra)

# Cierres por agencia

In [28]:

df_muestra = df_agencias.copy()
num = 24178000
print("Agencia: "+ str(num))
df_muestra = df_agencias[df_agencias['agencia'] == num].tail()
if df_muestra.empty:
    print("No se encuentran cierres para la agencia")
else:
    df_muestra.drop_duplicates(subset='monto',keep='last',inplace=True)
    print(df_muestra)

Agencia: 24178000
No se encuentran cierres para la agencia


# Cuadratura de abonos

In [ ]:
muestraex = df_mes[df_mes['abonos'].astype(int) == 49180]
muestrarut = df_mes[df_mes['descripcion'].str.contains('') | df_mes['descripcion'].str.contains('15184281-5')]
muestra = pd.concat([muestraex,muestrarut])
print(muestra)


Empty DataFrame
Columns: [fecha, sucursal, operacion, descripcion, cargos, abonos, ind]
Index: []


In [ ]:
df_abonos = df_abonos.drop(columns=['mcca'])
df_abonos.to_excel('C:\\Users\\' + username + '\\Documents\\Py\\JPNB\\GENERADOS\\Cuadratura AG. ' + str(num) + '.xlsx',
            header=True,
            index=False)

In [20]:
df_mes.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\JPNB\\GENERADOS\\Reporte mes ' + fecha_actual + '.csv',
            sep=';',
            header=False,
            index=False,
            date_format = date_formatexport)

In [ ]:
ndigitos = 6
num = 111660
print("Monto consultado: "+ str(num))
df_ag = df_agencias[df_agencias['mcca'].apply(lambda x:
         str(x).zfill(len(str(num)))[-ndigitos:] ==
         str(num).zfill(len(str(num)))[-ndigitos:])]['agencia']# .tail()
if df_ag.empty:
   print("N/A - No se encuentra agencia alguna vinculada al monto.")
else:
   print(df_ag)

Exportar

In [ ]:
tipos_fecha = df_mes['fecha'].unique()
tipos_sucursal = df_mes['sucursal'].unique()
tipos_operacion = df_mes['operacion'].unique()
tipos_descripcion = df_mes['descripcion'].unique()
tipos_agencia = df_mes['agencia'].unique()